## Defining the grammar


In [1]:
# Define the production rules of the grammar
grammar = {
    "E" : ["TĒ"],
    "Ē" : ["+TĒ", ""],
    "T" : ["FŤ"],
    "Ť": ["*FŤ", ""],
    "F" : ["(E)", "i"]
}

terminals = {'+', '*', 'i', '(', ')'}
non_terminals = ['E', 'Ē', 'T', 'Ť', 'F']
start_symbol = 'E'

## Get the First() of a Non-terminal

The `getFirst()` method takes in a non-terminal symbol and a set, populated the provided set with the corresponding First() terminals and returns the populated set

**Note🔴**: Empty strings have been programatically represented by `""`

In [2]:
# Get the First() of a non terminal and store the result in first
def getFirst(non_terminal, first):
    if non_terminal not in grammar:
      raise Exception("No production rule for this non terminal found")


    productions = grammar[non_terminal]
    for production in productions:
      if production == "":
        first.add("")
        continue
      if production[0] in terminals:
        first.add(production[0])
        continue
      else:
        # If the first symbol of a production is a non terminal, recursively find its first()
        getFirst(production[0], first)

    return first



### Populate the First() table
Getting First symbols for all non terminals

In [3]:
first = {}
for non_terminal in non_terminals:
  fst = set()
  getFirst(non_terminal, fst)
  first[non_terminal] = fst

print(first)

{'E': {'(', 'i'}, 'Ē': {'', '+'}, 'T': {'(', 'i'}, 'Ť': {'', '*'}, 'F': {'(', 'i'}}


## Get the Follow() of a Non-Terminal

#### Computing rules
```
1) FOLLOW(S) = { $ }   // where S is the starting Non-Terminal

2) If A -> pBq is a production, where p, B and q are any grammar symbols,
   then everything in FIRST(q)  except Є is in FOLLOW(B).

3) If A->pB is a production, then everything in FOLLOW(A) is in FOLLOW(B).

4) If A->pBq is a production and FIRST(q) contains Є,
   then FOLLOW(B) contains { FIRST(q) – Є } U FOLLOW(A)
   ```

   Rules from [GeeksforGeeks](https://www.geeksforgeeks.org/follow-set-in-syntax-analysis/)

In [4]:
follow = {}
for nt in non_terminals:
  follow[nt] = set()

def get_follow(non_terminal):
  '''
  2 cases:
  1) NT is at the end -> Follow += Follow(LHS)
  2) NT is not at the end:
    2.1) next symbol is NT -> Follow += First(next symbol):
      2.1.1 First(next symbol contains "") -> Follow += Follow(LHS)
    2.2) next symbol is T -> Follow += T
  '''
  if non_terminal == start_symbol:
    follow[non_terminal].add("$")
  for lhs in grammar:
    for production in grammar[lhs]:
      for i, char in enumerate(production):
        if char == non_terminal:
          if i == len(production) - 1:                              # Means that the NT is at the end of the production -> Add Follow of LHS
            follow[non_terminal].update(follow[lhs])                # Add follow of LHS to follow of NT
          else:
            for next_char in production[i+1:]:
              if next_char in terminals:
                follow[non_terminal].add(next_char)                 # If next character is a terminal, add the terminal itself
                break

              follow[non_terminal].update(first[next_char]-{""})    # Add First(next_char) to current NT Follow, excluding the empty string
              if '' not in first[next_char]:
                break                                               # If there's no empty string in First(next_char) end
              else:
                follow[non_terminal].update(follow[lhs])            # Add follow of LHS to follow of NT if First(next_char) contains empty string



### Populate the Follow() table

In [5]:
for nt in non_terminals:
  get_follow(nt)

print(follow)

{'E': {'$', ')'}, 'Ē': {'$', ')'}, 'T': {'$', ')', '+'}, 'Ť': {'$', ')', '+'}, 'F': {'*', '$', ')', '+'}}


## Displaying First and Follow Table

In [7]:
import pandas as pd
from tabulate import tabulate

data = []

for nt in non_terminals:
  data.append(
      {"Non-terminal": nt,
       "First": first[nt],
       "Follow": follow[nt]
       }
      )

df = pd.DataFrame(data)

# displaying the First and Follow table
print(tabulate(df, headers = 'keys', tablefmt = 'fancy_grid', showindex=False))

╒════════════════╤════════════╤══════════════════════╕
│ Non-terminal   │ First      │ Follow               │
╞════════════════╪════════════╪══════════════════════╡
│ E              │ {'(', 'i'} │ {'$', ')'}           │
├────────────────┼────────────┼──────────────────────┤
│ Ē              │ {'', '+'}  │ {'$', ')'}           │
├────────────────┼────────────┼──────────────────────┤
│ T              │ {'(', 'i'} │ {'$', ')', '+'}      │
├────────────────┼────────────┼──────────────────────┤
│ Ť              │ {'', '*'}  │ {'$', ')', '+'}      │
├────────────────┼────────────┼──────────────────────┤
│ F              │ {'(', 'i'} │ {'*', '$', ')', '+'} │
╘════════════════╧════════════╧══════════════════════╛


## Building LL(1) table

In [8]:
ll1 = {}
for non_terminal in non_terminals:
  ll1.update({non_terminal: {}})

grammar = {
    "E" : ["TĒ"],
    "Ē" : ["+TĒ", ""],
    "T" : ["FŤ"],
    "Ť": ["*FŤ", ""],
    "F" : ["(E)", "i"]
}

for lhs in grammar:
  for production in grammar[lhs]:
    if production == "":
      # Place production in follow of LHS
      table_entries = {}
      for terminal in follow[lhs]:
        table_entries[terminal] = {lhs: production}
      ll1[lhs] |= table_entries
      ll1.update()
    else:
      #Place production in first of RHS
      rhs_first_symbol = production[0]
      if rhs_first_symbol in terminals:                   # If first symbol of RHS is a terminal, place production on that terminal
        ll1[lhs][rhs_first_symbol] = {lhs: production}
      else:
        table_entries = {}
        for terminal in first[lhs]:                       # If first symbol of RHS is a non-terminal, place production on the First() of that non-terminal
          table_entries[terminal] = {lhs: production}
        ll1[lhs] |= table_entries

print(ll1)

{'E': {'(': {'E': 'TĒ'}, 'i': {'E': 'TĒ'}}, 'Ē': {'+': {'Ē': '+TĒ'}, '$': {'Ē': ''}, ')': {'Ē': ''}}, 'T': {'(': {'T': 'FŤ'}, 'i': {'T': 'FŤ'}}, 'Ť': {'*': {'Ť': '*FŤ'}, '$': {'Ť': ''}, ')': {'Ť': ''}, '+': {'Ť': ''}}, 'F': {'(': {'F': '(E)'}, 'i': {'F': 'i'}}}


### Displaying Final LL(1) table

In [9]:
data = []

def getProductionFromDict(production):
  # If no production exists, return empty string
  if production == '':
    return ''
  else:
    # Production given in the form of a dictionary, therefore extract lhs anf rhs
    lhs, rhs = next(iter( production.items() ))
    return (f'{lhs} -> {rhs if rhs else "ε" }')

for nt in non_terminals:
  productions = {}
  productions.update({'Non-terminal' : nt})
  for t in terminals:
    production = getProductionFromDict(ll1.get(nt).get(t, ''))
    productions.update({t : production})
  productions.update({'$' : getProductionFromDict(ll1.get(nt).get('$', ''))})

  data.append(
      # {"Non-terminal": nt},
      productions
      )


df = pd.DataFrame(data)

# displaying the First and Follow table
print(tabulate(df, headers = 'keys', tablefmt = 'fancy_grid', showindex=False))

╒════════════════╤══════════╤═════════╤══════════╤════════╤══════════╤════════╕
│ Non-terminal   │ (        │ i       │ +        │ )      │ *        │ $      │
╞════════════════╪══════════╪═════════╪══════════╪════════╪══════════╪════════╡
│ E              │ E -> TĒ  │ E -> TĒ │          │        │          │        │
├────────────────┼──────────┼─────────┼──────────┼────────┼──────────┼────────┤
│ Ē              │          │         │ Ē -> +TĒ │ Ē -> ε │          │ Ē -> ε │
├────────────────┼──────────┼─────────┼──────────┼────────┼──────────┼────────┤
│ T              │ T -> FŤ  │ T -> FŤ │          │        │          │        │
├────────────────┼──────────┼─────────┼──────────┼────────┼──────────┼────────┤
│ Ť              │          │         │ Ť -> ε   │ Ť -> ε │ Ť -> *FŤ │ Ť -> ε │
├────────────────┼──────────┼─────────┼──────────┼────────┼──────────┼────────┤
│ F              │ F -> (E) │ F -> i  │          │        │          │        │
╘════════════════╧══════════╧═════════╧═